In [1]:
from skimage.io import imread
import matplotlib.pyplot as plt
import math
import os
from skimage.io import imread
from skimage.color import rgb2grey
from skimage.measure import approximate_polygon
from skimage.filters import gaussian
from skimage.color import rgb2grey
from skimage.filters import threshold_adaptive, threshold_yen, threshold_local
from scipy import ndimage as ndi
from skimage.measure import find_contours
from skimage.draw import polygon_perimeter
import numpy as np
from skimage import transform as tf
import skimage
from skimage.feature import blob_dog, blob_log, blob_doh
from scipy.ndimage import gaussian_filter

sudoku_original = rgb2grey(imread('http://1.bp.blogspot.com/-7y7o4rdh_G8/TVWnhbfmPrI/AAAAAAAAAHA/ZW_s56iEZMo/s1600/sudoku%2B001.jpg'))
sudoku_original = skimage.transform.resize(sudoku_original,(700,700))
plt.axis('off')
plt.imshow(sudoku_original)
plt.show()

/home/wolf/.local/lib/python2.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/wolf/.local/lib/python2.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


<Figure size 640x480 with 1 Axes>

In [2]:
#all utilities
def clear_noisy_borders(six):
    #vertical first
    for i in range(20):
        if((six[:,i] == 0).sum() > 80):
            six[:,i] = (six[:,i]*0)+1

    for i in range(110,128):
        if((six[:,i] == 0).sum() > 80):
            six[:,i] = (six[:,i]*0)+1

    #horizontal now
    for i in range(20):
        if((six[i,:] == 0).sum() > 80):
            six[i,:] = (six[i,:]*0)+1

    for i in range(110,128):
        if((six[i,:] == 0).sum() > 80):
            six[i,:] = (six[i,:]*0)+1
    
    return six

def area_of_rectangle(Xmin, Xmax, Ymin, Ymax):
    return ((Ymax-Ymin)*(Xmax-Xmin))

def find_areas_bound_by_contours(contours):
    areas_bound_by_contours = []
    for index, contour in enumerate(contours):
        Xmin = np.min(contour[:,0])
        Xmax = np.max(contour[:,0])
        Ymin = np.min(contour[:,1])
        Ymax = np.max(contour[:,1])
        areas_bound_by_contours.append([index,area_of_rectangle(Xmin, Xmax, Ymin, Ymax)])
        
    return areas_bound_by_contours

def get_highest_area(areas_bound_by_contours):
    highest_area = [0,0]
    for areas in areas_bound_by_contours:
        if areas[1] > highest_area[1]:
            highest_area = areas
    return highest_area

def fill_holes(six):
    return ndi.binary_fill_holes(six)

#ref:https://stackoverflow.com/questions/1211212/how-to-calculate-an-angle-from-three-points
def findAngle(p0,p1,p2):
    b = (p1[0]-p0[0])**2 + (p1[1]-p0[1])**2
    a = (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2
    c = (p2[0]-p0[0])**2 + (p2[1]-p0[1])**2
    return math.degrees(math.acos( (a+b-c) / math.sqrt(4*a*b) ))


def pre_process_block(box):
    #box = warped[row*100:row*100+100,col*100:col*100+100].copy()
    box = skimage.transform.resize(box,(128,128))
    box = gaussian_filter(box, 1)
    box_binary_adaptive = threshold_local(box, block_size=99)
    binary = box <= box_binary_adaptive
    box = skimage.util.invert(binary)*1
    box = clear_noisy_borders(box)
    contours = find_contours(box, level=0.8)
    areas_bound_by_contours = find_areas_bound_by_contours(contours)

    for area in areas_bound_by_contours:
        threshold = 0.33
        percentage_of_area = area[1]/(128*128)
        if percentage_of_area<threshold:
            cont = contours[area[0]].astype(int)
            Xmin = np.min(cont[:,0])
            Xmax = np.max(cont[:,0])
            Ymin = np.min(cont[:,1])
            Ymax = np.max(cont[:,1])
            box[Xmin:Xmax, Ymin:Ymax] = 1

    return box

In [3]:
contours = find_contours(sudoku_original, 0.8)

In [4]:
# fig, ax = plt.subplots(figsize=(20,20))
# ax.axis("off")
# ax.imshow(sudoku_original, cmap="gray")
# for i in range(len(contours)-1):
#     ax.plot(contours[i][:, 1], contours[i][:, 0], linewidth=2)

In [5]:
len(contours)

9239